In [72]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('../data/ai_vs_human_code_dataset.csv')
# code id varmaan turha (?) + mitenköhän tuo code_snippets variable hyödynnetään?
# olisko mahdollista tehdä malli, jossa syötettäisiin itse koodinpätkiä ja malli predictaisi onko human vai ai?
# logistista regressiota olisi mahdollista käyttää kyseiseen toteutukseen. Samalla voitaisiin analysoida (coefficients) ja tutkia mitkä ovat tärkeitä variableja siinä että onko koodi ai vai human

# EHDOTUS MITEN DATAN VOISI KÄSITELLÄ

# Preprocess the code snippets
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['code_snippet'])

# Convert the CSR matrix to a dense array
X_tfidf_dense = X_tfidf.toarray()

# Create a DataFrame from the dense array
X_tfidf_df = pd.DataFrame(X_tfidf_dense, columns=tfidf.get_feature_names_out())

# Standardize the numerical features
numerical_features = df.drop(columns=['code_snippet', 'ai_generated', 'code_id'])
scaler = StandardScaler()
numerical_features_scaled = scaler.fit_transform(numerical_features)

# Create a DataFrame from the standardized numerical features
numerical_features_df = pd.DataFrame(numerical_features_scaled, columns=numerical_features.columns)

# Combine TF-IDF features with the standardized numerical features
X_combined = pd.concat([X_tfidf_df, numerical_features_df.reset_index(drop=True)], axis=1)

# Define the target variable
y = df['ai_generated']

df

,code_id,code_snippet,code_length,num_functions,num_classes,num_variables,num_comments,comment_density,num_imports,use_of_recursion,use_of_lambda,num_loops,complexity_score,execution_time,ai_generated
0,04bd1e86,import random\r\ndef random_numbers(n): return...,86,1,0,0,0,0.0,1,0,0,1,2,0.60,1
1,03020544,"x = lambda a, b: a + b\r\nprint(x(3, 4))",37,0,0,1,0,0.0,0,0,1,0,0,1.83,1
2,a6a08557,"def fib(n): a, b = 0, 1\r\n for _ in range(n):...",69,1,0,2,0,0.0,0,0,0,1,2,0.51,1
3,edce6fd4,def factorial(n): return 1 if n == 0 else n * ...,62,1,0,0,0,0.0,0,1,0,0,3,10.93,1
4,03020544,"x = lambda a, b: a + b\r\nprint(x(3, 4))",37,0,0,1,0,0.0,0,0,1,0,0,3.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,a8ee5b6a,"import statistics\r\nvalues = [10, 20, 30]\r\n...",70,0,0,1,0,0.0,1,0,0,0,0,1.47,0
1496,7147f334,"def add_numbers(a, b):\r\n return a + b\r\n...",64,1,0,0,0,0.0,0,0,0,0,1,1.14,0
1497,0e7f4f30,"def fibonacci(n):\r\n sequence = [0, 1]\r\n...",142,1,0,1,0,0.0,0,0,0,1,2,2.22,0
1498,b48fc10d,import random\r\nrandom.seed(42)\r\ndef gen_ra...,102,1,0,0,0,0.0,1,0,0,1,2,2.30,0


In [73]:
import re
import time

def ensure_newline_after_functions(code_snippet):
    # Pattern to match function definitions without a newline after them
    pattern = r'(\bdef\s+\w+\s*\(.*\):)([^\n])'

    # Add a newline after function definitions if not present
    modified_code = re.sub(pattern, r'\1\n\2', code_snippet)

    return modified_code

j = 0

for i in range(len(df['code_snippet'])):
    j += 1
    sample = df['code_snippet'][i]
    function_pattern = r'def\s+(\w+)\s*\((\w+)\):'
    match = re.search(function_pattern, sample)

    if match:
        function_name = match.group(1)
        parameter_name = match.group(2)

        # Replace parameter n with 10
        modified_code = re.sub(r'\b' + parameter_name + r'\b', '10', sample)

        # Add function call to the end of the code snippet
        function_call = f'\nresult = {function_name}(300)'
        sample += function_call
        sample = ensure_newline_after_functions(sample)
        start_time = time.perf_counter_ns()
        exec (sample)
        end_time = time.perf_counter_ns()
    else:
        exec(sample)



7
7
7
7
2.0
7
2.0
7
7
2.0
2.0
2.0
2.0
7
2.0
7
2.0
2.0
2.0
7
2.0
7
2.0
7
2.0
7
7
2.0
2.0
7
7
7
7
2.0
2.0
7
7
2.0
2.0
2.0
7
2.0
2.0
2.0
7
7
7
7
2.0
7
7
7
7
7
7
7
2.0
2.0
2.0
2.0
7
2.0
7
2.0
7
2.0
2.0
7
2.0
7
7
2.0
2.0
2.0
2.0
2.0
7
2.0
2.0
7
2.0
2.0
7
7
2.0
2.0
2.0
2.0
2.0
2.0
7
7
2.0
2.0
2.0
7
7
7
7
2.0
7
2.0
2.0
2.0
2.0
7
2.0
7
7
7
7
7
2.0
7
7
7
2.0
7
2.0
2.0
7
7
2.0
7
2.0
2.0
2.0
2.0
2.0
2.0
7
2.0
2.0
2.0
7
2.0
7
7
7
2.0
7
2.0
7
2.0
2.0
7
7
2.0
7
2.0
7
2.0
2.0
7
7
2.0
7
2.0
7
7
7
2.0
7
2.0
2.0
2.0
7
7
2.0
2.0
2.0
7
7
7
2.0
7
2.0
2.0
7
7
2.0
7
7
7
2.0
7
2.0
7
7
2.0
7
2.0
7
7
7
7
7
2.0
2.0
2.0
2.0
7
7
7
7
2.0
7
7
7
2.0
2.0
7
2.0
2.0
7
2.0
7
2.0
7
7
7
7
7
7
2.0
7
7
2.0
2.0
2.0
7
7
2.0
2.0
7
7
7
2.0
7
7
2.0
2.0
2.0
2.0
7
2.0
2.0
2.0
2.0
2.0
7
7
7
2.0
2.0
2.0
2.0
2.0
2.0
7
7
2.0
2.0
2.0
7
2.0
7
7
7
7
2.0
7
7
7
7
7
7
2.0
7
7
2.0
7
7
2.0
2.0
2.0
2.0
7
7
7
2.0
7
2.0
2.0
2.0
7
2.0
7
7
2.0
7
7
2.0
7
7
2.0
7
2.0
7
2.0
2.0
7
7
7
20
7
7
20
20
20
20
7
20
20
20
7
20
20
20
7
20
20
20
20
20
7
20
7
20
